In [2]:
"""
Andres Trujillo
4/20/2025
SNHU
Prof. Helen
Module Six Milestone
"""

#from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc
from dash import html
from dash import Dash
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#My file
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username   = 'aacuser'
password   = '123snhu'
host       = 'localhost'
port       = 27017
database   = 'AAC'
collection = 'animals'

shelter = AnimalShelter(username, password, host, port, database, collection)

#Debug animal shelter object, database connection
#print(shelter)
#shelter.debug()

#Update values or else it will return all documents
query = None
projection = None
df = pd.DataFrame.from_records(shelter.read(query, projection))

#Dropping first column
df.drop(columns=['_id'],inplace=True)


## Debug dataframe
#print(len(df.to_dict(orient='records')))
#print(df.columns)


###########################
# Dashboard Layout / View #
###########################
"""
html.Div(className='buttonRow',style={'display':'flex'},children=[
            html.Button(id='submit-button', n_clicks=0, children='Cats'),
            html.Button(id='submit-button-two', n_clicks=0,children='Dogs')]),
"""
app = Dash('Animal Rescue and Search')

app.layout = html.Div([html.Div(id = 'hidden-div', style = {'display':'none'}),
    html.Div([
        html.Center([
            html.A(html.Img(src='assets/Grazioso_Salvare_Logo.png', style={'height': '80px'}),
                href='https://www.snhu.edu',
                target='_blank'  
            ),
            html.H1('Animal Search and Rescue Dashboard', style={'fontWeight': 'bold'})
        ])
    ]),
    html.Hr(),
    html.Div(dcc.RadioItems(id='my-radio',
                   options=[{'label':'Water Rescue','value':'water'},
                            {'label':'Mountain Rescue','value':'mountain'},
                            {'label':'Disaster Rescue','value':'disaster'},
                            {'label':'Reset','value':'reset'}],value='all')),
    html.Hr(),
    html.Div(id='output-div'),
    dash_table.DataTable(id='datatable-id',
        selected_rows=[0],
        columns=[{"name": i,"id": i,"deletable": False,"selectable": True} for i in df.columns],
        data = df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        page_action="native",
        page_current=0,
        page_size=10,
        style_header={'whiteSpace':'normal','height':'auto','textAlign':'center','fontWeight':'bold'}
    ),
    html.Br(),
    html.Hr(),
    html.Div([
    html.Div(id='map-id', style={'width': '50%', 'padding': '10px'}),
    html.Div(dcc.Graph(id='chart-id'), style={'width': '50%', 'padding': '10px'})
], style={'display': 'flex', 'flexDirection': 'row'}),
    html.Hr(),
    html.Div(
        children=[
            html.Img(
                src="https://avatars.githubusercontent.com/u/57707223?v=4&size=128",
                style={'borderRadius': '50%', 'width': '64px', 'height': '64px', 'marginRight': '10px'}
            ),
            html.Div([
                html.P("Andres Trujillo", style={'margin': 0}),
                html.P("Unique Id: 068017c6-8c82-4e36-9af9-bcc7cf94f9c9", style={'margin': 0, 'fontSize': '12px', 'color': '#666'})
            ])
        ],
        style={
            'display': 'flex',
            'alignItems': 'center',
            'justifyContent': 'center',
            'marginTop': '40px',
            'padding': '10px',
            'borderTop': '1px solid #ccc'
        }
    )
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(
    Output('chart-id', 'figure'),
    Input('datatable-id', 'derived_virtual_data')
)
def update_chart(data):
    dff = pd.DataFrame(data) if data else df

    # Count occurrences of each breed
    breed_counts = dff['breed'].value_counts().reset_index()
    breed_counts.columns = ['breed', 'count']

    # Horizontal bar chart
    fig = px.bar(
        breed_counts,
        x='count',
        y='breed',
        orientation='h',
        title='Breed Count',
        labels={'count': 'Number of Animals', 'breed': 'Breed'}
    )

    fig.update_layout(
        yaxis={'categoryorder': 'total ascending'},
        plot_bgcolor='white',
        xaxis_tickformat='0'
    )

    return fig

@app.callback(
    [Output('datatable-id', 'data'),
    Output('datatable-id', 'style_data')],
    [Input('my-radio', 'value'),
     Input('datatable-id', 'derived_virtual_data')]  
)
def update_output(selected, viewData):

    # If the "Reset" option is selected (i.e., load all documents)
    if selected == 'all':
        style = {'backgroundColor': '#D2F3FF'}
        return df.to_dict('records'),style  # Return the full dataset
    
    style = {'backgroundColor':'#FF0000','color':'#FFFFFF'}

    # If specific filter option is selected
    if selected == 'water':
        target_breeds = ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']
        
        # Filter the dataframe based on client specifications
        dff = pd.DataFrame.from_dict(viewData) if viewData else df
        dff = dff[
            (dff['breed'].isin(target_breeds)) & 
            (dff['sex_upon_outcome'] == 'Intact Female') & 
            (dff['age_upon_outcome_in_weeks'].apply(pd.to_numeric, errors='coerce').between(26, 156,inclusive='both'))
        ]
        return dff.to_dict('records'), style
    
    if selected == 'mountain':
        target_breeds = ['German Shepherd','Alaskan Malamute','Old English Sheepdog','Siberian Husky','Rottweiler']
        
        #Filter 
        dff = pd.DataFrame.from_dict(viewData) if viewData else df
        dff = dff[
            (dff['breed'].isin(target_breeds)) &
            (dff['sex_upon_outcome'] == 'Intact Male') &
            (dff['age_upon_outcome_in_weeks'].apply(pd.to_numeric, errors='coerce').between(26,156,inclusive='both'))
        ]
        return dff.to_dict('records'),style
    
    if selected == 'disaster':
        target_breeds = ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']
        
        #Filter
        dff = pd.DataFrame.from_dict(viewData) if viewData else df
        dff = dff[
            (dff['breed'].isin(target_breeds)) &
            (dff['sex_upon_outcome'] == 'Intact Male') &
            (dff['age_upon_outcome_in_weeks'].apply(pd.to_numeric, errors='coerce').between(26,156,inclusive='both'))
        ]
        return dff.to_dict('records'), style
    
    if selected == 'reset':
        style = {'backgroundColor': '#D2F3FF'}
        return df.to_dict('records'), style


#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])

def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


#This callback will create map

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

def update_map(viewData, index):
    
    if not viewData or index is None or len(index) == 0:
        return html.Div([html.P("No data loaded yet.")])
    dff = pd.DataFrame.from_dict(viewData)
    
    if index is None:
        row = 0
    else:
        row = index[0]
    
    lat  = dff.iloc[row]['location_lat'] #column 13
    long = dff.iloc[row]['location_long'] #column 14
    animalBreed = dff.iloc[row]['breed'] #4 Column
    animalName = dff.iloc[row]['name'] #9 Column

    return [dl.Map(
        style={'width': '100%', 'height': '500px'},
        center=[lat, long],
        zoom=10,
        children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, long], children=[
                dl.Tooltip(animalBreed),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(animalName)
                ])
            ])
        ]
    )]

# Run the app server
app.run()